In [2]:
import glob
import pandas as pd

file = glob.glob("train/train.tsv")
pd.read_csv("train/train.tsv", delimiter='\t')
data_list = []
data_list.append(pd.read_csv("train/train.tsv", delimiter='\t', index_col=0))
df = pd.concat(data_list)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb

X = df.drop('Cover_Type', axis=1)
y = df['Cover_Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBMのモデルを設定
model = lgb.LGBMClassifier(num_leaves=31, learning_rate=0.05, n_estimators=100)

# モデルをトレーニング
model.fit(X_train, y_train)

# テストデータに対する予測
y_pred = model.predict(X_test)

# 予測結果の評価
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))#適合率の表示

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001395 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2263
[LightGBM] [Info] Number of data points in the train set: 232404, number of used features: 53
[LightGBM] [Info] Start training from score -1.010496
[LightGBM] [Info] Start training from score -0.717265
[LightGBM] [Info] Start training from score -2.788987
[LightGBM] [Info] Start training from score -5.318327
[LightGBM] [Info] Start training from score -4.125123
[LightGBM] [Info] Start training from score -3.500427
[LightGBM] [Info] Start training from score -3.348253
Accuracy: 82.66%


In [37]:
# テストデータを読み込む
df_test = pd.read_csv("train/test.tsv", delimiter='\t')#tsv形式のため区切り文字はタブ
df_test = df_test.rename(columns={df_test.columns[0]: 'id'})#df_testの０列目がid用なのでidと名付け
# テストデータから予測に必要な特徴量を選択
X_test_competition = df_test.drop('id', axis=1)  # idは予測には不要なので除外

# 先のセルで学習済みモデルを使用して予測を行う
y_pred_competition = model.predict(X_test_competition)

# 予測結果をデータフレームに保存
results_df = pd.DataFrame({
    'ID': df_test['id'],  # テストデータのID列
    'Predicted_Cover_Type': y_pred_competition
})
results_df
# ファイルに書き出し（ヘッダーなし、インデックスなし）
results_df.to_csv('sample_submit.csv', index=False, header=False)